Este cuadernillo combina la información proveniente de varias tablas en un único conjunto de datos consolidado.  
Durante el proceso se eliminan las columnas irrelevantes y se aplican técnicas de anonimización para proteger la información sensible.  

Nota: Este cuadernillo debe ejecutarse únicamente después de haber corrido `prepro.ipynb`.

En este bloque se importan las librerías necesarias para el análisis. Se utiliza pandas para la manipulación y análisis de datos en forma de tablas, mientras que os se emplea para la gestión de rutas y archivos dentro del sistema operativo.

In [1]:
# Block 1 — Imports
# Load core libraries for data manipulation and filesystem paths
import pandas as pd
import numpy as np
import os

En este bloque se listan todas las tablas disponibles y se seleccionan únicamente las cuatro principales: Atención, Triage, Evolución e Interconsultas, que son las que se utilizarán para realizar el proceso de unión de datos.

In [2]:
# Block 2 — Define available and selected files
# List all possible tables and select only the four main ones for the join

all_files = [
    'Atencion',
    'Triage',
    'Evolucion',
    'Analisis',
    'Procedimientos',
    'Consultas',
    'Diagnosticos',
    'Formulacion',
    'Formatos'
]

# Use only the four essential tables for the dataset
files = [
    'Atencion',
    'Triage',
    'Evolucion',
    'InterConsultas'
]

En este bloque se cargan exclusivamente las cuatro tablas seleccionadas desde la carpeta de datos para trabajar con ellas en el proceso de integración.

In [3]:
# Block 3 — Load CSV files
# Load only the four selected tables into a dictionary of DataFrames

df = {}  # dictionary to store the DataFrames

data_dir = r"C:\Users\wilmerbelza\Documents\Prediction model"

for file in files:
    csv_path = os.path.join(data_dir, f"{file}.csv")
    df[file] = pd.read_csv(csv_path, low_memory=False)
    print(f"Loaded: {csv_path}")
    print(df[file].head())

Loaded: C:\Users\wilmerbelza\Documents\Prediction model\Atencion.csv
   NumeroDocumento TipoDocumento     Genero  EdadAtencion  Ingreso  \
0  01032023                  AS    Femenino            34        1   
1  01072711799               CC    Femenino            27        1   
2  0748972                   PA    Femenino            49        3   
3  083880036                 PA   Masculino            69        1   
4  1000001527                CC   Masculino            22        2   

         FechaAdmision          FechaEgreso SalidaClinica  \
0  2023-03-01 01:42:50  1753-01-01 00:00:00             N   
1  2023-08-17 18:15:08  2023-08-18 00:07:06             S   
2  2023-06-01 04:22:24  2023-06-02 07:46:57             N   
3  2023-02-07 14:30:04  2023-02-07 14:39:27             N   
4  2023-04-18 12:41:16  1753-01-01 00:00:00             N   

  FechaAnulacionIngreso  
0   2023-03-02 13:46:44  
1   1753-01-01 00:00:00  
2   1753-01-01 00:00:00  
3   2023-02-07 14:39:27  
4   2023-04-2

Idea:

- Atencion: Es el primer registro de un paciente en la clínica. Contiene la marca de tiempo inicial y final.  
  La llave primaria = NumeroDocumento, TipoDocumento, Ingreso.  
  *Ingreso* es secuencial para cada paciente.

- Triage: Es el siguiente paso en la clínica, pero tiene menos filas que Atencion porque no todos los pacientes realizan Triage (pueden abandonar antes).  
  Como se hará una unión interna (inner join), los pacientes sin Triage serán descartados.  
  La llave primaria = NumeroDocumento, TipoDocumento, Ingreso, Folio.  
  *Folio* es secuencial para cada paciente.

- Evolucion: Contiene todas las interacciones con los pacientes durante su estancia. Puede tener varias filas para la misma llave.  
  Solo se debe considerar el primer registro (mínima marca de tiempo inmediatamente después del Triage).

- InterConsultas: Se maneja con la misma lógica de Evolucion.

En este bloque se definen funciones auxiliares que permiten identificar columnas con nombres variables en las diferentes tablas y, además, normalizar los valores asociados al género y a la clasificación de triage para mantener consistencia en el conjunto de datos.

In [4]:
# Block 4 — Utility functions and normalizers
# Define helpers to detect columns with variable names and normalize categorical values

# Keys that uniquely identify each episode
keys = ['NumeroDocumento', 'TipoDocumento', 'Ingreso']

def pick_col(df_, candidates):
    """Return the first existing column from a list of candidates, or None if none are found."""
    for c in candidates:
        if c in df_.columns:
            return c
    return None

def normalize_gender(x):
    """Normalize gender values to 'M' or 'F'."""
    if pd.isna(x):
        return np.nan
    x = str(x).strip().upper()
    if x.startswith("M"):  # M, MALE, MASCULINO
        return "M"
    if x.startswith("F"):  # F, FEMALE, FEMENINO
        return "F"
    return np.nan

def normalize_triage(x):
    """Normalize triage values to integers 1–5 if possible."""
    if pd.isna(x):
        return np.nan
    x = str(x).strip().upper()
    for i in range(1, 6):
        if str(i) in x or x.endswith(str(i)) or x == f"NIVEL {i}" or x == f"CLASE {i}":
            return i
    return np.nan

En este bloque se transforma la columna de tiempo del triage (FechaHoraAtencion) al formato de fecha y se conserva únicamente el primer registro de cada paciente, de modo que cada episodio quede representado por una sola fila en esta etapa.

In [5]:
# Block 5 — Deduplicate TRIAGE (FechaHoraAtencion -> FechaTriage)
tri = df['Triage'].copy()

tri_time_col = pick_col(tri, [
    'FechaHoraAtencion',   # your real column
    'FechaTriage', 'Fecha Hora Triage', 'FechaHoraTriage'
])
if tri_time_col is None:
    raise KeyError("No triage timestamp column found in TRIAGE.")

tri[tri_time_col] = pd.to_datetime(tri[tri_time_col], errors='coerce')
tri = tri.rename(columns={tri_time_col: 'FechaTriage'})

triage_min = (tri.dropna(subset=['FechaTriage'])
                .sort_values(keys + ['FechaTriage'])
                .drop_duplicates(subset=keys, keep='first'))
assert triage_min.duplicated(keys).sum() == 0

En este bloque se estandarizan las columnas de la tabla Atención, cambiando el nombre de FechaAdmision por FechaRegistro. Posteriormente se ejecuta un inner join con la tabla de Triage, ya depurada, para integrar ambos pasos iniciales del flujo y conservar únicamente a los pacientes que cuentan con registros en las dos etapas.

In [6]:
# Block 6 — Standardize ATENCION and inner-join with triage_min
atn = df['Atencion'].copy()

adm_col  = pick_col(atn, ['FechaAdmision', 'Fecha Admision'])
eg_col   = pick_col(atn, ['FechaEgreso', 'Fecha Egreso'])
exit_col = pick_col(atn, ['SalidaClinica', 'Salida Clínica'])
gen_col  = pick_col(atn, ['Genero','Género','Sexo'])
edad_col = pick_col(atn, ['EdadAtencion','Edad Atencion','Edad'])

if adm_col is None:
    raise KeyError("No admission column found in ATENCION.")

rename_map = {adm_col: 'FechaRegistro'}
if eg_col:   rename_map[eg_col]   = 'FechaEgreso'
if exit_col: rename_map[exit_col] = 'SalidaClinica'
if gen_col:  rename_map[gen_col]  = 'Genero'
if edad_col: rename_map[edad_col] = 'EdadAtencion'

atn = atn[keys + [c for c in [adm_col, eg_col, exit_col, gen_col, edad_col] if c]].rename(columns=rename_map)

# cast dates
for c in ['FechaRegistro','FechaEgreso']:
    if c in atn.columns:
        atn[c] = pd.to_datetime(atn[c], errors='coerce')

# inner join with one-row-per-episode triage
tri_cols = [c for c in keys + ['FechaTriage','ClasificacionTriage','MotivoConsulta'] if c in triage_min.columns]
dfj = atn.merge(triage_min[tri_cols], on=keys, how='inner')
assert dfj.duplicated(keys).sum() == 0

En este bloque se identifica la primera evolución registrada de cada paciente y se incorpora al conjunto de datos mediante un left join, lo que permite conservar también los episodios que no presentan evolución.

In [7]:
# Block 7 — First EVOLUCION per patient
# Keep only the earliest evolution timestamp and merge with the main dataset

ev = df['Evolucion'].copy()

# Detect evolution timestamp column
evo_time_col = pick_col(ev, [
    'FechaHoraEvolucion',
    'FechaHoraAtencion',
    'Fecha Hora Evolucion'
])
if evo_time_col is None:
    raise KeyError("No timestamp column found in EVOLUCION.")

# Convert to datetime
ev[evo_time_col] = pd.to_datetime(ev[evo_time_col], errors='coerce')

# One row per episode: earliest evolution
first_eval = (
    ev.dropna(subset=[evo_time_col])
      .sort_values(keys + [evo_time_col])
      .drop_duplicates(subset=keys, keep='first')
      .rename(columns={evo_time_col: 'FechaPrimeraEvolucion'})
)

# Left join with the dataset (dfj from Atención+Triage)
dfj = dfj.merge(first_eval[keys + ['FechaPrimeraEvolucion']], on=keys, how='left')

# Sanity check: no duplicated episodes
assert dfj.duplicated(keys).sum() == 0, "Duplicated episodes after Evolucion join."

En este bloque se selecciona la primera interconsulta de cada paciente y se incorpora al conjunto de datos mediante un left join, preservando igualmente los episodios que no registran interconsultas.

In [8]:
# Block 8 — First INTERCONSULTAS per patient
# Keep only the earliest interconsultation timestamp and merge with the main dataset

ic = df['InterConsultas'].copy()

# Detect interconsultation timestamp column
ic_time_col = pick_col(ic, [
    'FechaHoraOrden',
    'FechaHoraAtencion',
    'Fecha Hora Interconsulta',
    'FechaHoraInterconsulta'
])
if ic_time_col is None:
    # If no timestamp column exists, create an empty result and merge later
    first_ic = pd.DataFrame(columns=keys + ['FechaPrimeraInterconsulta'])
else:
    # Convert to datetime
    ic[ic_time_col] = pd.to_datetime(ic[ic_time_col], errors='coerce')

    # One row per episode: earliest interconsultation
    first_ic = (
        ic.dropna(subset=[ic_time_col])
          .sort_values(keys + [ic_time_col])
          .drop_duplicates(subset=keys, keep='first')
          .rename(columns={ic_time_col: 'FechaPrimeraInterconsulta'})
    )

# Left join with the main dataset (dfj from previous blocks)
merge_cols = [c for c in keys + ['FechaPrimeraInterconsulta'] if c in first_ic.columns]
dfj = dfj.merge(first_ic[merge_cols], on=keys, how='left')

# Sanity check: no duplicated episodes after join
assert dfj.duplicated(keys).sum() == 0, "Duplicated episodes after Interconsultas join."

En este bloque se calculan estadísticas de cobertura para identificar cuántos episodios cuentan con evolución, interconsulta, ambos registros o al menos uno de los dos.

In [9]:
# Block 9 — Coverage metrics
# Compute how many episodes have evolution, interconsultation, both, or at least one

n_base = len(dfj)

has_evo = dfj['FechaPrimeraEvolucion'].notna() if 'FechaPrimeraEvolucion' in dfj.columns else pd.Series([False]*n_base)
has_ic  = dfj['FechaPrimeraInterconsulta'].notna() if 'FechaPrimeraInterconsulta' in dfj.columns else pd.Series([False]*n_base)

n_evo  = has_evo.sum()
n_ic   = has_ic.sum()
n_both = (has_evo & has_ic).sum()
n_any  = (has_evo | has_ic).sum()

def pct(x): 
    return f"{(100*x/n_base):.2f}%" if n_base else "0.00%"

print(f"Episodes (Atencion+Triage): {n_base:,}")
print(f"With first Evolucion: {n_evo:,} ({pct(n_evo)})")
print(f"With first Interconsulta: {n_ic:,} ({pct(n_ic)})")
print(f"With both: {n_both:,} ({pct(n_both)})")
print(f"With either: {n_any:,} ({pct(n_any)})")

Episodes (Atencion+Triage): 57,850
With first Evolucion: 46,114 (79.71%)
With first Interconsulta: 23,640 (40.86%)
With both: 23,618 (40.83%)
With either: 46,136 (79.75%)


En este bloque se establece la primera evaluación como la fecha más temprana entre evolución e interconsulta. Además, se filtran los episodios para garantizar un orden temporal coherente en el recorrido del paciente, cumpliendo la secuencia Registro < Triage < Evaluación.

In [10]:
# Block 10 — Define earliest evaluation and filter temporal consistency
# Compute FechaPrimeraEvaluacion and keep only episodes with Registro < Triage < Evaluacion

# Define earliest valid evaluation
dfj['FechaPrimeraEvaluacion'] = dfj[['FechaPrimeraEvolucion', 'FechaPrimeraInterconsulta']].min(axis=1)

# Build mask for temporal consistency
mask_ok = (
    dfj['FechaRegistro'].notna() &
    dfj['FechaTriage'].notna() &
    dfj['FechaPrimeraEvaluacion'].notna() &
    (dfj['FechaTriage'] > dfj['FechaRegistro']) &
    (dfj['FechaPrimeraEvaluacion'] > dfj['FechaTriage'])
)

# Apply filter
dfj = dfj[mask_ok].copy()

# Diagnostic
print(f"Final dataset after temporal filter: {len(dfj)} episodes")

Final dataset after temporal filter: 46135 episodes


En este bloque se lleva a cabo una inspección preliminar de las tablas principales y del conjunto de datos unificado, revisando tamaños, columnas relevantes, primeras filas y rangos de fechas con el fin de validar que el proceso de unión se haya realizado de manera consistente.

In [11]:
# Block 11 — Quick visual inspection of tables and unified dataset
# Show shapes, columns, heads, and basic date ranges to validate the join

def show_overview(name, df_):
    print(f"\n=== {name} ===")
    print(f"shape: {df_.shape}")
    print("columns:", list(df_.columns))
    print(df_.head(3))

# Source tables (raw/trimmed) and unified dataset
show_overview("ATENCION (raw)", df['Atencion'])
show_overview("TRIAGE (raw)", df['Triage'])
show_overview("EVOLUCION (raw)", df['Evolucion'])
show_overview("INTERCONSULTAS (raw)", df['InterConsultas'])
show_overview("JOIN (dfj)", dfj)

# Quick date ranges (only if columns exist)
def date_range(df_, col):
    if col in df_.columns:
        s = pd.to_datetime(df_[col], errors='coerce')
        return s.min(), s.max(), s.notna().mean()
    return None

print("\n--- Date ranges (min, max, non-null %) ---")
for col in ["FechaRegistro", "FechaTriage", "FechaPrimeraEvolucion", "FechaPrimeraInterconsulta", "FechaPrimeraEvaluacion"]:
    rng = date_range(dfj, col)
    if rng:
        dmin, dmax, pnn = rng
        print(f"{col:>26}: {dmin}  ->  {dmax}   | non-null: {pnn:.2%}")

# Quick categorical snapshot for triage (if present)
if "ClasificacionTriage" in dfj.columns:
    print("\n--- ClasificacionTriage: top frequencies ---")
    print(dfj["ClasificacionTriage"].value_counts(dropna=False).head(10))


=== ATENCION (raw) ===
shape: (63303, 9)
columns: ['NumeroDocumento', 'TipoDocumento', 'Genero', 'EdadAtencion', 'Ingreso', 'FechaAdmision', 'FechaEgreso', 'SalidaClinica', 'FechaAnulacionIngreso']
   NumeroDocumento TipoDocumento    Genero  EdadAtencion  Ingreso  \
0  01032023                  AS   Femenino            34        1   
1  01072711799               CC   Femenino            27        1   
2  0748972                   PA   Femenino            49        3   

         FechaAdmision          FechaEgreso SalidaClinica  \
0  2023-03-01 01:42:50  1753-01-01 00:00:00             N   
1  2023-08-17 18:15:08  2023-08-18 00:07:06             S   
2  2023-06-01 04:22:24  2023-06-02 07:46:57             N   

  FechaAnulacionIngreso  
0   2023-03-02 13:46:44  
1   1753-01-01 00:00:00  
2   1753-01-01 00:00:00  

=== TRIAGE (raw) ===
shape: (57850, 10)
columns: ['NumeroDocumento', 'TipoDocumento', 'Genero', 'EdadAtencion', 'Ingreso', 'FechaAdmision', 'Folio', 'FechaHoraAtencion', 'Cla

En este bloque se verifican los duplicados según la clave definida, se identifican valores nulos en campos críticos y se calculan los intervalos de tiempo entre Registro y Triage, entre Triage y Evaluación, y entre Registro y Evaluación.

In [13]:
# Block 12 — Data quality checks and time deltas
# Check duplicates, critical nulls, and compute key time intervals (in minutes)

# 1) Duplicates by episode keys
dup = dfj.duplicated(['NumeroDocumento','TipoDocumento','Ingreso']).sum()
print(f"Duplicated episodes in dfj: {dup}")

# 2) Critical nulls snapshot
critical_cols = ['FechaRegistro','FechaTriage','FechaPrimeraEvaluacion']
print("\nCritical nulls (%):")
for c in critical_cols:
    if c in dfj.columns:
        pct_null = 100 * (1 - dfj[c].notna().mean())
        print(f" - {c}: {pct_null:.2f}%")
    else:
        print(f" - {c}: column not present")

# 3) Time deltas (minutes)
def minutes(a, b):
    return (b - a).dt.total_seconds() / 60.0

dfj['min_registro_a_triage'] = minutes(dfj['FechaRegistro'], dfj['FechaTriage'])
dfj['min_triage_a_eval']     = minutes(dfj['FechaTriage'], dfj['FechaPrimeraEvaluacion'])
dfj['min_registro_a_eval']   = minutes(dfj['FechaRegistro'], dfj['FechaPrimeraEvaluacion'])

# 4) Descriptive stats
print("\nDescriptive stats (minutes):")
for c in ['min_registro_a_triage','min_triage_a_eval','min_registro_a_eval']:
    if c in dfj.columns:
        s = dfj[c].dropna()
        if not s.empty:
            print(f"{c}: count={s.size:,}, mean={s.mean():.1f}, median={s.median():.1f}, "
                  f"p90={s.quantile(0.90):.1f}, p95={s.quantile(0.95):.1f}")
        else:
            print(f"{c}: no data")

Duplicated episodes in dfj: 0

Critical nulls (%):
 - FechaRegistro: 0.00%
 - FechaTriage: 0.00%
 - FechaPrimeraEvaluacion: 0.00%

Descriptive stats (minutes):
min_registro_a_triage: count=46,135, mean=12.3, median=8.2, p90=23.9, p95=31.4
min_triage_a_eval: count=46,135, mean=103.4, median=40.4, p90=154.9, p95=200.8
min_registro_a_eval: count=46,135, mean=115.7, median=52.8, p90=168.5, p95=215.3


En este bloque se seleccionan y organizan las columnas que formarán parte del datalet, definiendo su orden antes de aplicar el proceso de anonimización.

In [17]:
# Block 13 — Final column selection (no anonymization, no export)
# Keep only the columns needed for the datalet and fix their order

cols_final = [c for c in [
    'NumeroDocumento','TipoDocumento','Ingreso',
    'Genero','EdadAtencion',
    'FechaRegistro','FechaTriage','ClasificacionTriage','MotivoConsulta',
    'FechaPrimeraEvolucion','FechaPrimeraInterconsulta','FechaPrimeraEvaluacion',
    'FechaEgreso','SalidaClinica',
    'min_registro_a_triage','min_triage_a_eval','min_registro_a_eval'
] if c in dfj.columns]

final = dfj[cols_final].copy()
print("Final shape (no anonymization):", final.shape)
print(final.head(5))

Final shape (no anonymization): (46135, 17)
   NumeroDocumento TipoDocumento  Ingreso     Genero  EdadAtencion  \
1  01072711799               CC         1   Femenino            27   
3  1000002426                CC         1  Masculino            26   
4  1000002426                CC         2  Masculino            26   
5  1000002426                CC         3  Masculino            26   
6  1000002572                CC         4   Femenino            22   

        FechaRegistro         FechaTriage  ClasificacionTriage  \
1 2023-08-17 18:15:08 2023-08-17 18:31:41                    3   
3 2023-04-04 12:16:32 2023-04-04 12:21:24                    2   
4 2023-09-22 11:16:51 2023-09-22 11:20:51                    3   
5 2023-09-23 11:01:40 2023-09-23 11:03:53                    3   
6 2023-01-04 09:10:34 2023-01-04 09:15:22                    3   

                                      MotivoConsulta FechaPrimeraEvolucion  \
1  Se realiza atención del paciente, con medidas ...   2023-

En este bloque se genera un identificador anónimo para cada episodio y se eliminan los identificadores directos de los pacientes, garantizando la confidencialidad de la información.

In [18]:
# Block 14 — Anonymization (no export)
# Create a hashed episode identifier and drop direct IDs

from hashlib import sha256

SALT = "replace_with_a_secure_random_salt"  # <-- set a strong random salt before running

def make_hash(row):
    concat = f"{row['NumeroDocumento']}|{row['TipoDocumento']}|{row['Ingreso']}"
    return sha256((SALT + concat).encode('utf-8')).hexdigest()[:16]

# Build anonymous id
final['id_anon'] = final.apply(make_hash, axis=1)

# Drop direct identifiers
final = final.drop(columns=['NumeroDocumento','TipoDocumento'], errors='ignore')

# Reorder columns (put id_anon first)
ordered = ['id_anon','Ingreso'] + [c for c in final.columns if c not in ['id_anon','Ingreso']]
final = final[ordered]

print("Final shape (with id_anon):", final.shape)

Final shape (with id_anon): (46135, 16)


En este bloque se valida que la anonimización se haya aplicado correctamente y se presenta una vista preliminar con las columnas clave y las primeras filas del conjunto de datos.

In [19]:
# Block 15 — Post-anonymization QA + Preview
# Ensure anonymization is correct and show a readable preview

# 1) QA on anonymization
assert 'id_anon' in final.columns, "id_anon was not created."
assert final['id_anon'].isna().sum() == 0, "id_anon contains nulls."
assert final['id_anon'].duplicated().sum() == 0, "id_anon has duplicates."

# 2) Temporal sanity checks (non-negative minutes)
for c in ['min_registro_a_triage','min_triage_a_eval','min_registro_a_eval']:
    if c in final.columns:
        assert (final[c].dropna() >= 0).all(), f"Negative values found in {c}"

print("Post-anonymization QA passed.")

# 3) Compact preview: show key columns for the first 10 rows
preview_cols = [c for c in [
    'id_anon','Ingreso',
    'FechaRegistro','FechaTriage','FechaPrimeraEvaluacion',
    'min_registro_a_triage','min_triage_a_eval','min_registro_a_eval',
    'ClasificacionTriage','SalidaClinica'
] if c in final.columns]

print("\n=== Preview (first 10 rows) ===")
print(final[preview_cols].head(10))

Post-anonymization QA passed.

=== Preview (first 10 rows) ===
             id_anon  Ingreso       FechaRegistro         FechaTriage  \
1   6124828209892124        1 2023-08-17 18:15:08 2023-08-17 18:31:41   
3   9294009b87eac4f1        1 2023-04-04 12:16:32 2023-04-04 12:21:24   
4   63cd4a4d6b899314        2 2023-09-22 11:16:51 2023-09-22 11:20:51   
5   8a3c5faefad7b7eb        3 2023-09-23 11:01:40 2023-09-23 11:03:53   
6   bda4010562c40190        4 2023-01-04 09:10:34 2023-01-04 09:15:22   
7   12961e128b446ee1        4 2023-08-25 18:46:00 2023-08-25 18:57:21   
8   9796b471e986fbf0        4 2023-01-01 03:45:05 2023-01-01 03:55:11   
10  00851f0b3c3935b5        6 2023-05-25 16:43:03 2023-05-25 16:44:33   
11  0bc65fa35937b4fb        7 2023-10-02 14:48:09 2023-10-02 14:54:20   
12  1e643c5d226524f9        8 2023-10-08 13:28:11 2023-10-08 13:31:25   

   FechaPrimeraEvaluacion  min_registro_a_triage  min_triage_a_eval  \
1     2023-08-17 20:10:06              16.550000          98.4

En este bloque se guarda el datalet anonimizado en la ubicación indicada, utilizando la función to_csv con la ruta completa especificada.

In [21]:
# Block 16 — Export final anonymized datalet
# Save directly to the requested path (replace SALT before running!)

final.to_csv(r'C:\Users\wilmerbelza\Documents\Prediction model\mh.csv', index=False)
print("Saved anonymized datalet to: C:\\Users\\wilmerbelza\\Documents\\Prediction model\\mh.csv")

Saved anonymized datalet to: C:\Users\wilmerbelza\Documents\Prediction model\mh.csv


En este bloque se imprime un resumen compacto que muestra el tamaño final del conjunto de datos y las coberturas obtenidas, con el fin de dar cierre al pipeline.

In [23]:
# Block 17 — Final pipeline summary
# Compact end-of-pipeline recap: size, coverage, and date ranges

import numpy as np
print("\n=== PIPELINE SUMMARY ===")
print("Rows in final:", len(final))
print("Columns in final:", len(final.columns))

def cov(col):
    return final[col].notna().mean() if col in final.columns else np.nan

print("Coverage — Evolucion: "
      f"{cov('FechaPrimeraEvolucion'):.2%} | Interconsulta: "
      f"{cov('FechaPrimeraInterconsulta'):.2%} | Evaluacion: "
      f"{cov('FechaPrimeraEvaluacion'):.2%}")

def dr(col):
    if col in final.columns:
        s = pd.to_datetime(final[col], errors='coerce')
        return f"{s.min()} → {s.max()} ({s.notna().mean():.2%} non-null)"
    return "n/a"

for col in ['FechaRegistro','FechaTriage','FechaPrimeraEvaluacion']:
    print(f"{col}: {dr(col)}")


=== PIPELINE SUMMARY ===
Rows in final: 46135
Columns in final: 16
Coverage — Evolucion: 99.95% | Interconsulta: 51.24% | Evaluacion: 100.00%
FechaRegistro: 2023-01-01 00:49:09 → 2023-12-31 23:38:44 (100.00% non-null)
FechaTriage: 2023-01-01 01:01:52 → 2023-12-31 23:44:18 (100.00% non-null)
FechaPrimeraEvaluacion: 2023-01-01 01:09:52 → 2024-01-13 18:08:45 (100.00% non-null)
